In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import models, transforms
from torchvision.datasets import ImageFolder

In [ ]:
# Step 3: Define Image Transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

In [ ]:
 #Step 4: Load Datasets
train_dir = '/content/drive/MyDrive/Deep_learning/TrimDataset/Dataset/Train'
val_dir = '/content/drive/MyDrive/Deep_learning/TrimDataset/Dataset/Validation'
test_dir = '/content/drive/MyDrive/Deep_learning/TrimDataset/Dataset/Test'

train_data = ImageFolder(train_dir, transform=transform)
val_data = ImageFolder(val_dir, transform=transform)
test_data = ImageFolder(test_dir, transform=transform)

train_loader = DataLoader(train_data, batch_size=16, shuffle=True)
val_loader = DataLoader(val_data, batch_size=16, shuffle=False)
test_loader = DataLoader(test_data, batch_size=16, shuffle=False)

In [ ]:
# Step 5: Define resnet Model for Deepfake Detection
class CNN_Deepfake(nn.Module):
    def __init__(self, output_size=1):
        super(CNN_Deepfake, self).__init__()

        # Load a pre-trained ResNet model
        resnet = models.resnet18(pretrained=True)
        self.feature_extractor = nn.Sequential(*list(resnet.children())[:-1])  # Remove last FC layer

        # Fully connected layer for classification
        self.fc = nn.Linear(512, output_size)
        self.sigmoid = nn.Sigmoid()  # For binary classification

    def forward(self, x):
        # CNN feature extraction
        x = self.feature_extractor(x)  # Shape: (batch_size, 512, 1, 1)
        x = x.view(x.size(0), -1)      # Flatten to (batch_size, 512)
        x = self.fc(x)                 # Fully connected layer
        return self.sigmoid(x)

In [ ]:
# Define Loss Function and Optimizer
model = CNN_Deepfake(output_size=1)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Step 7: Training Loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for images, labels in train_loader:
        # Prepare labels for binary classification
        labels = labels.float() # Remove unsqueeze(1)

        # Forward pass
        outputs = model(images).squeeze()
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

print("Training complete")

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch [1/10], Loss: 0.0213
Epoch [2/10], Loss: 0.1698
Epoch [3/10], Loss: 0.1885
Epoch [4/10], Loss: 0.0062
Epoch [5/10], Loss: 0.3369
Epoch [6/10], Loss: 0.0227
Epoch [7/10], Loss: 0.0214
Epoch [8/10], Loss: 0.1717
Epoch [9/10], Loss: 0.5216
Epoch [10/10], Loss: 0.0293
Training complete


In [ ]:
torch.save(model.state_dict(), '/content/drive/MyDrive/DL/cnn_model.pth')

NameError: name 'model' is not defined

In [ ]:
from PIL import Image
import torchvision.transforms as transforms

# Replace with the name of your uploaded image
image_path = '/content/drive/MyDrive/Deep_learning/TrimDataset/Dataset/Validation/Fake/fake_2038.jpg'
image = Image.open(image_path)

# Apply the same transformation
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])
image = transform(image).unsqueeze(0)  # Add batch dimension
# Load the model
model = CNN_Deepfake(output_size=1)
model.load_state_dict(torch.load('/content/drive/MyDrive/DL/cnn_model.pth'))
model.eval()
with torch.no_grad():
    output = model(image).item()  # Get the prediction value
    prediction = "Real" if output > 0.5 else "Fake"

    print(f"Prediction: {prediction} (Probability of being Real: {output:.4f})")


<ipython-input-10-a7d508545bda>:16: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('/content/drive/MyDrive/DL/cnn_model.pth'))


Prediction: Fake (Probability of being Real: 0.0000)
